In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.getcwd() + '/src/analytic/database'))
from db_api import *

In [2]:
db = db_api()

In [3]:
stats = db.get_items('stats')
for stat in stats:
    print stat.name

QUEUE_LEN
ITEMS_PROCESSED
MEMORY_ALLOCATED
NUM_STATES


In [9]:
msus = db.get_items('msus')
for msu in msus:
    print 'ID: ' + str(msu.id) + ', NAME: ' + msu.msu_type.name

ID: 1, NAME: SOCKET
ID: 2, NAME: WS_READ
ID: 3, NAME: WS_READ
ID: 4, NAME: WS_HTTP
ID: 5, NAME: WS_REGEX
ID: 6, NAME: WS_REGEX
ID: 7, NAME: WS_REGEX
ID: 8, NAME: WS_WRITE
ID: 9, NAME: WS_REGEX_ROUTE
ID: 10, NAME: WS_REGEX_ROUTE
ID: 11, NAME: WS_REGEX
ID: 12, NAME: WS_REGEX
ID: 13, NAME: WS_REGEX
ID: 14, NAME: WS_REGEX
ID: 15, NAME: WS_REGEX
ID: 16, NAME: WS_REGEX


In [10]:
ts = db.get_msu_timeseries(msus[0])
for t in ts:
    print t.statistic.name

QUEUE_LEN
ITEMS_PROCESSED
MEMORY_ALLOCATED
NUM_STATES
